In [1]:
import pandas as pd

path = "germany_cities_stadtrecht_2023.csv"  
df = pd.read_csv(path)

df.head()


,state,city,city_designation,area_km2,postal_code,population_total,male,female,per_km2
0,Berlin,Berlin,NaN,891.12,10178,3782202,1860115,1922087,4 244
1,Hamburg,Hamburg,Freie und Hansestadt,755.09,20038,1910160,936740,973420,2 530
2,Bayern,München,Landeshauptstadt,310.70,80313,1510378,734925,775453,4 861
3,Nordrhein-Westfalen,Köln,NaN,405.02,50667,1087353,527728,559625,2 685
4,Hessen,Frankfurt am Main,NaN,248.31,60311,775790,382226,393564,3 124


In [2]:
print("Shape:", df.shape)
df.columns


Shape: (2059, 9)


Index(['state', 'city', 'city_designation', 'area_km2', 'postal_code',
       'population_total', 'male', 'female', 'per_km2'],
      dtype='object')

In [3]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2059 entries, 0 to 2058
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   state             2059 non-null   object 
 1   city              2059 non-null   object 
 2   city_designation  156 non-null    object 
 3   area_km2          2059 non-null   float64
 4   postal_code       2059 non-null   int64  
 5   population_total  2059 non-null   int64  
 6   male              2059 non-null   int64  
 7   female            2059 non-null   int64  
 8   per_km2           2059 non-null   object 
dtypes: float64(1), int64(4), object(4)
memory usage: 144.9+ KB


In [4]:
df.isna().sum().sort_values(ascending=False).head(20)


city_designation    1903
state                  0
city                   0
area_km2               0
postal_code            0
population_total       0
male                   0
female                 0
per_km2                0
dtype: int64

In [5]:
df.duplicated().sum()


np.int64(0)

In [6]:
df.describe(include="all").T


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
state,2059,16,Bayern,317,NaN,NaN,NaN,NaN,NaN,NaN,NaN
city,2059,2051,Waldenburg,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
city_designation,156,71,GKSt,29,NaN,NaN,NaN,NaN,NaN,NaN,NaN
area_km2,2059.0,NaN,NaN,NaN,73.225085,62.998129,0.45,32.36,59.51,93.23,891.12
postal_code,2059.0,NaN,NaN,NaN,50875.831472,30417.329555,1067.0,24572.0,52396.0,76446.5,99994.0
population_total,2059.0,NaN,NaN,NaN,30218.810588,115322.631109,254.0,6312.5,12783.0,25218.0,3782202.0
male,2059.0,NaN,NaN,NaN,14856.225352,56595.930968,125.0,3183.5,6393.0,12367.0,1860115.0
female,2059.0,NaN,NaN,NaN,15362.585236,58730.935212,129.0,3172.5,6446.0,12750.5,1922087.0
per_km2,2059,866,76,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
[c for c in df.columns if "ein" in c.lower() or "bev" in c.lower() or "pop" in c.lower() or "stadt" in c.lower() or "lat" in c.lower() or "lon" in c.lower()]


['population_total']

In [8]:
def to_int(x):
    if pd.isna(x): 
        return pd.NA
    s = str(x).strip()
    s = s.replace(".", "").replace(",", "")  # binlik ayırıcıları temizle
    return pd.to_numeric(s, errors="coerce")

# ÖRNEK: population kolonu adını bulduktan sonra buraya yazacağız
# df["population"] = df["population_raw"].apply(to_int)


In [9]:
df_clean = df[
    ["state", "city", "population_total", "area_km2", "per_km2"]
].copy()


In [10]:
df_clean = df_clean.rename(columns={
    "state": "state",
    "city": "city",
    "population_total": "population",
    "area_km2": "area_km2",
    "per_km2": "density_per_km2"
})


In [11]:
df_clean = df_clean[df_clean["population"] >= 30000]
df_clean.shape


(406, 5)

In [12]:
df_clean["demand_weight"] = (
    df_clean["population"] / df_clean["population"].sum()
)


In [16]:
df_clean.isna().sum()


state              0
city               0
population         0
area_km2           0
density_per_km2    0
demand_weight      0
dtype: int64

In [17]:
df_clean.head()


,state,city,population,area_km2,density_per_km2,demand_weight
0,Berlin,Berlin,3782202,891.12,4 244,0.087483
1,Hamburg,Hamburg,1910160,755.09,2 530,0.044182
2,Bayern,München,1510378,310.70,4 861,0.034935
3,Nordrhein-Westfalen,Köln,1087353,405.02,2 685,0.025151
4,Hessen,Frankfurt am Main,775790,248.31,3 124,0.017944


In [18]:
df_clean.describe()


,population,area_km2,demand_weight
count,4.060000e+02,406.000000,406.000000
mean,1.064864e+05,103.935296,0.002463
std,2.451340e+05,82.759111,0.005670
min,3.007000e+04,14.650000,0.000696
25%,3.832950e+04,56.980000,0.000887
50%,4.954200e+04,83.015000,0.001146
75%,8.265475e+04,122.437500,0.001912
max,3.782202e+06,891.120000,0.087483


# combining 2 dataset

In [19]:
coords = pd.read_csv("Deutschland_Cities.csv")
coords.head()


,city,lat,lng,country,iso2,admin_name,capital,population,population_proper
0,Berlin,52.5167,13.3833,Germany,DE,Berlin,primary,3644826.0,3644826.0
1,Hamburg,53.5500,10.0000,Germany,DE,Hamburg,admin,1841179.0,1841179.0
2,Munich,48.1372,11.5755,Germany,DE,Bavaria,admin,1471508.0,1471508.0
3,Cologne,50.9422,6.9578,Germany,DE,North Rhine-Westphalia,NaN,1085664.0,1085664.0
4,Frankfurt,50.1136,8.6797,Germany,DE,Hesse,minor,753056.0,753056.0


In [20]:
df_clean["city_std"] = df_clean["city"].str.strip().str.lower()
coords["city_std"] = coords["city"].str.strip().str.lower()


In [24]:
coords.columns


Index(['city', 'lat', 'lng', 'country', 'iso2', 'admin_name', 'capital',
       'population', 'population_proper', 'city_std'],
      dtype='object')

In [26]:
df_geo = df_clean.merge(
    coords[["city_std", "lat", "lng"]],
    on="city_std",
    how="inner"
)

df_geo = df_geo.rename(columns={"lat": "latitude", "lng": "longitude"})

print("Merged shape:", df_geo.shape)
df_geo.head()


Merged shape: (347, 9)


,state,city,population,area_km2,density_per_km2,demand_weight,city_std,latitude,longitude
0,Berlin,Berlin,3782202,891.12,4 244,0.087483,berlin,52.5167,13.3833
1,Hamburg,Hamburg,1910160,755.09,2 530,0.044182,hamburg,53.5500,10.0000
2,Baden-Württemberg,Stuttgart,633484,207.33,3 055,0.014653,stuttgart,48.7761,9.1775
3,Nordrhein-Westfalen,Düsseldorf,631217,217.41,2 903,0.014600,düsseldorf,51.2311,6.7724
4,Sachsen,Leipzig,619879,297.80,2 082,0.014338,leipzig,51.3500,12.3833


In [27]:
matched_ratio = df_geo.shape[0] / df_clean.shape[0]
print("Match ratio:", matched_ratio)

missing = df_clean.loc[~df_clean["city_std"].isin(coords["city_std"]), ["state","city"]]
missing.head(20)


Match ratio: 0.854679802955665


,state,city
2,Bayern,München
3,Nordrhein-Westfalen,Köln
4,Hessen,Frankfurt am Main
13,Bayern,Nürnberg
30,Sachsen-Anhalt,Halle (Saale)
44,Rheinland-Pfalz,Ludwigshafen am Rhein
45,Niedersachsen,Oldenburg (Oldenburg)
46,Nordrhein-Westfalen,Mülheim an der Ruhr
57,Hessen,Offenbach am Main
82,Brandenburg,Cottbus/Chóśebuz


In [28]:
import re

def normalize_city(s: str) -> str:
    if pd.isna(s):
        return s
    s = str(s).strip().lower()

    # Alman karakterleri normalize
    s = (s.replace("ä", "a")
           .replace("ö", "o")
           .replace("ü", "u")
           .replace("ß", "ss"))

    # Parantez içini sil: "Halle (Saale)" -> "halle"
    s = re.sub(r"\s*\(.*?\)\s*", " ", s)

    # Slash sonrası alternatif ismi sil: "cottbus/chos..." -> "cottbus"
    s = s.split("/")[0]

    # Noktalama ve fazla boşluk temizle
    s = re.sub(r"[^a-z0-9\s-]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()

    return s


In [29]:
import re

def normalize_city(s: str) -> str:
    if pd.isna(s):
        return s
    s = str(s).strip().lower()

    # Alman karakterleri normalize
    s = (s.replace("ä", "a")
           .replace("ö", "o")
           .replace("ü", "u")
           .replace("ß", "ss"))

    # Parantez içini sil: "Halle (Saale)" -> "halle"
    s = re.sub(r"\s*\(.*?\)\s*", " ", s)

    # Slash sonrası alternatif ismi sil: "cottbus/chos..." -> "cottbus"
    s = s.split("/")[0]

    # Noktalama ve fazla boşluk temizle
    s = re.sub(r"[^a-z0-9\s-]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()

    return s


In [31]:
print("coords columns:", coords.columns.tolist())
print("df_clean columns:", df_clean.columns.tolist())


coords columns: ['city', 'lat', 'lng', 'country', 'iso2', 'admin_name', 'capital', 'population', 'population_proper', 'city_std']
df_clean columns: ['state', 'city', 'population', 'area_km2', 'density_per_km2', 'demand_weight', 'city_std']


In [33]:
import re

def normalize_city(s):
    if pd.isna(s):
        return s
    s = str(s).strip().lower()
    s = (s.replace("ä", "a")
           .replace("ö", "o")
           .replace("ü", "u")
           .replace("ß", "ss"))
    s = re.sub(r"\s*\(.*?\)\s*", " ", s)   # parantez içini sil
    s = s.split("/")[0]                   # slash sonrası sil
    s = re.sub(r"[^a-z0-9\s-]", " ", s)   # noktalama temizle
    s = re.sub(r"\s+", " ", s).strip()
    return s

df_clean["city_key"] = df_clean["city"].apply(normalize_city)
coords["city_key"]   = coords["city"].apply(normalize_city)


In [34]:
df_clean[["city","city_key"]].head()
coords[["city","city_key"]].head()


,city,city_key
0,Berlin,berlin
1,Hamburg,hamburg
2,Munich,munich
3,Cologne,cologne
4,Frankfurt,frankfurt


In [35]:
df_geo2 = df_clean.merge(
    coords[["city_key", "lat", "lng"]],
    on="city_key",
    how="inner"
).rename(columns={"lat":"latitude", "lng":"longitude"})

print("New merged shape:", df_geo2.shape)
print("New match ratio:", df_geo2.shape[0] / df_clean.shape[0])
df_geo2.head()


New merged shape: (373, 10)
New match ratio: 0.9187192118226601


,state,city,population,area_km2,density_per_km2,demand_weight,city_std,city_key,latitude,longitude
0,Berlin,Berlin,3782202,891.12,4 244,0.087483,berlin,berlin,52.5167,13.3833
1,Hamburg,Hamburg,1910160,755.09,2 530,0.044182,hamburg,hamburg,53.5500,10.0000
2,Baden-Württemberg,Stuttgart,633484,207.33,3 055,0.014653,stuttgart,stuttgart,48.7761,9.1775
3,Nordrhein-Westfalen,Düsseldorf,631217,217.41,2 903,0.014600,düsseldorf,dusseldorf,51.2311,6.7724
4,Sachsen,Leipzig,619879,297.80,2 082,0.014338,leipzig,leipzig,51.3500,12.3833


In [36]:
missing2 = df_clean.loc[~df_clean["city_key"].isin(coords["city_key"]), ["state","city","city_key"]]
missing2.head(30)


,state,city,city_key
2,Bayern,München,munchen
3,Nordrhein-Westfalen,Köln,koln
4,Hessen,Frankfurt am Main,frankfurt am main
13,Bayern,Nürnberg,nurnberg
44,Rheinland-Pfalz,Ludwigshafen am Rhein,ludwigshafen am rhein
46,Nordrhein-Westfalen,Mülheim an der Ruhr,mulheim an der ruhr
57,Hessen,Offenbach am Main,offenbach am main
84,Baden-Württemberg,Esslingen am Neckar,esslingen am neckar
120,Brandenburg,Brandenburg an der Havel,brandenburg an der havel
130,Hessen,Rüsselsheim am Main,russelsheim am main


In [37]:
missing2 = df_clean.loc[~df_clean["city_key"].isin(coords["city_key"]), ["state","city","city_key"]]
print("Missing count:", len(missing2))
print("Missing ratio:", len(missing2)/len(df_clean))
missing2.head(20)


Missing count: 37
Missing ratio: 0.09113300492610837


,state,city,city_key
2,Bayern,München,munchen
3,Nordrhein-Westfalen,Köln,koln
4,Hessen,Frankfurt am Main,frankfurt am main
13,Bayern,Nürnberg,nurnberg
44,Rheinland-Pfalz,Ludwigshafen am Rhein,ludwigshafen am rhein
46,Nordrhein-Westfalen,Mülheim an der Ruhr,mulheim an der ruhr
57,Hessen,Offenbach am Main,offenbach am main
84,Baden-Württemberg,Esslingen am Neckar,esslingen am neckar
120,Brandenburg,Brandenburg an der Havel,brandenburg an der havel
130,Hessen,Rüsselsheim am Main,russelsheim am main


In [38]:
def normalize_city_v2(s: str) -> str:
    if pd.isna(s):
        return s
    s = str(s).strip().lower()

    # Alman karakterleri
    s = (s.replace("ä", "a")
           .replace("ö", "o")
           .replace("ü", "u")
           .replace("ß", "ss"))

    # kısaltmaları genişlet (çok kritik!)
    s = s.replace(" i.d.", " in der ").replace(" i.d", " in der ")
    s = s.replace(" a. d.", " an der ").replace(" a. d", " an der ")
    s = s.replace(" v. d.", " vor der ").replace(" v. d", " vor der ")
    s = s.replace(" b.", " bei ").replace(" b", " bei ")

    # parantez içi sil
    s = re.sub(r"\s*\(.*?\)\s*", " ", s)

    # slash sonrası at
    s = s.split("/")[0]

    # noktaları tamamen kaldır (OPf. gibi)
    s = s.replace(".", " ")

    # noktalama ve boşluk temizliği
    s = re.sub(r"[^a-z0-9\s-]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()

    return s


In [40]:
df_clean["city_key2"] = df_clean["city"].apply(normalize_city_v2)
coords["city_key2"]   = coords["city"].apply(normalize_city_v2)


In [41]:
df_clean["city_key2"] = df_clean["city"].apply(normalize_city_v2)
coords["city_key2"]   = coords["city"].apply(normalize_city_v2)


In [42]:
df_geo3 = df_clean.merge(
    coords[["city_key2", "lat", "lng"]],
    on="city_key2",
    how="inner"
).rename(columns={"lat":"latitude","lng":"longitude"})

print("Match ratio v2:", df_geo3.shape[0] / df_clean.shape[0])


Match ratio v2: 0.9187192118226601


In [43]:
facility_names = ["Berlin","Hamburg","München","Köln","Frankfurt am Main"]
facility_keys = [normalize_city_v2(x) for x in facility_names]

facilities = df_geo3[df_geo3["city_key2"].isin(facility_keys)].copy()
facilities[["city","state","latitude","longitude"]]

,city,state,latitude,longitude
0,Berlin,Berlin,52.5167,13.3833
1,Hamburg,Hamburg,53.5500,10.0000


In [44]:
cities = df_geo3[["city_key2", "city", "state", "latitude", "longitude", "population", "demand_weight"]].copy()
cities = cities.dropna(subset=["latitude", "longitude"]).drop_duplicates(subset=["city_key2"])
cities = cities.reset_index(drop=True)

N = len(cities)
print("N cities with coords:", N)
cities.head()


N cities with coords: 368


,city_key2,city,state,latitude,longitude,population,demand_weight
0,berlin,Berlin,Berlin,52.5167,13.3833,3782202,0.087483
1,hamburg,Hamburg,Hamburg,53.5500,10.0000,1910160,0.044182
2,stuttgart,Stuttgart,Baden-Württemberg,48.7761,9.1775,633484,0.014653
3,dusseldorf,Düsseldorf,Nordrhein-Westfalen,51.2311,6.7724,631217,0.014600
4,leipzig,Leipzig,Sachsen,51.3500,12.3833,619879,0.014338


In [57]:
import numpy as np
import pandas as pd

lat = np.radians(cities["latitude"].to_numpy())
lon = np.radians(cities["longitude"].to_numpy())

# R: radius of the world
R = 6371.0

dlat = lat[:, None] - lat[None, :]
dlon = lon[:, None] - lon[None, :]

a = np.sin(dlat/2)**2 + np.cos(lat[:, None]) * np.cos(lat[None, :]) * np.sin(dlon/2)**2
c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
D = R * c  # NxN distance matrix in km


In [46]:
iu = np.triu_indices(N, k=1)  # üst üçgen, diagonal hariç

dist_long = pd.DataFrame({
    "city_a": cities.loc[iu[0], "city"].to_numpy(),
    "city_b": cities.loc[iu[1], "city"].to_numpy(),
    "distance_km": D[iu]
})

dist_long.head(), dist_long.shape


(   city_a      city_b  distance_km
 0  Berlin     Hamburg   253.700404
 1  Berlin   Stuttgart   510.635691
 2  Berlin  Düsseldorf   475.624889
 3  Berlin     Leipzig   146.729853
 4  Berlin    Dortmund   419.931019,
 (67528, 3))

In [47]:
dist_long.to_csv("distance_all_pairs_uppertriangle.csv", index=False)


In [48]:
print(len(cities))


368


In [50]:
cities = df_geo3[["city_key2", "city", "state", "population", "demand_weight", "latitude", "longitude"]].copy()
cities = cities.dropna(subset=["latitude", "longitude"]).drop_duplicates(subset=["city_key2"])
cities = cities.reset_index(drop=True)

print("N:", len(cities))
cities.head()


N: 368


,city_key2,city,state,population,demand_weight,latitude,longitude
0,berlin,Berlin,Berlin,3782202,0.087483,52.5167,13.3833
1,hamburg,Hamburg,Hamburg,1910160,0.044182,53.5500,10.0000
2,stuttgart,Stuttgart,Baden-Württemberg,633484,0.014653,48.7761,9.1775
3,dusseldorf,Düsseldorf,Nordrhein-Westfalen,631217,0.014600,51.2311,6.7724
4,leipzig,Leipzig,Sachsen,619879,0.014338,51.3500,12.3833


In [51]:
cities.to_csv("clean_cities_with_population_coords.csv", index=False)


In [52]:
import numpy as np
import pandas as pd

lat = np.radians(cities["latitude"].to_numpy())
lon = np.radians(cities["longitude"].to_numpy())

R = 6371.0
dlat = lat[:, None] - lat[None, :]
dlon = lon[:, None] - lon[None, :]

a = np.sin(dlat/2)**2 + np.cos(lat[:, None]) * np.cos(lat[None, :]) * np.sin(dlon/2)**2
c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
D = R * c

N = len(cities)
iu = np.triu_indices(N, k=1)

dist_long = pd.DataFrame({
    "city_a": cities.loc[iu[0], "city"].to_numpy(),
    "city_b": cities.loc[iu[1], "city"].to_numpy(),
    "distance_km": D[iu]
})

print(dist_long.shape)
dist_long.head()


(67528, 3)


,city_a,city_b,distance_km
0,Berlin,Hamburg,253.700404
1,Berlin,Stuttgart,510.635691
2,Berlin,Düsseldorf,475.624889
3,Berlin,Leipzig,146.729853
4,Berlin,Dortmund,419.931019


In [53]:
dist_long.to_csv("distance_all_pairs_uppertriangle.csv", index=False)


In [54]:
print("Any negative distance?", (dist_long["distance_km"] < 0).any())
print("Any zero distance?", (dist_long["distance_km"] == 0).any())


Any negative distance? False
Any zero distance? False


In [55]:
# demand sum
cities["demand_weight"].sum()


np.float64(0.8663471812727377)

In [56]:
# distance basic check
dist_long["distance_km"].describe()


count    67528.000000
mean       280.100249
std        150.068241
min          1.789941
25%        162.906514
50%        274.190745
75%        386.312635
max        837.792251
Name: distance_km, dtype: float64

In [ ]:
şi